In [2]:
import scanpy as sc
import numpy as np 
import polars as pl
import anndata
import pandas as pd 

In [3]:
count = pd.read_csv('/work/home/cryoem666/xyf/temp/pycharm/Squidiff/data/log_normalised_counts.csv', index_col=0).transpose()
meta  = pl.read_csv(
    "/work/home/cryoem666/xyf/temp/pycharm/Squidiff/data/cell_metadata_cols.tsv",
    separator="\t",
    ignore_errors=True
).to_pandas()
adata_iPSC = anndata.AnnData(count)
adata_iPSC.obs=meta.copy()
adata_iPSC.layers["counts"] = adata_iPSC.X.copy()


/work/home/cryoem666/miniconda3/envs/my_state/lib/python3.12/functools.py:912: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


In [4]:
adata_iPSC.obs['day']

0        day1
1        day1
2        day1
3        day1
4        day1
         ... 
36039    day1
36040    day1
36041    day1
36042    day1
36043    day1
Name: day, Length: 36044, dtype: object

In [5]:
adata_iPSC.obs.columns

Index(['assigned', 'untitled', 'auxDir', 'cell_filter', 'cell_name',
       'compatible_fragment_ratio', 'day', 'donor', 'expected_format',
       'experiment', 'frag_dist_length', 'gc_bias_correct', 'is_cell_control',
       'is_cell_control_bulk', 'is_cell_control_control', 'library_types',
       'libType', 'log10_total_counts', 'log10_total_counts_endogenous',
       'log10_total_counts_ERCC', 'log10_total_counts_feature_control',
       'log10_total_counts_MT', 'log10_total_features',
       'log10_total_features_endogenous', 'log10_total_features_ERCC',
       'log10_total_features_feature_control', 'log10_total_features_MT',
       'mapping_type', 'mates1', 'mates2', 'n_alt_reads', 'n_total_reads',
       'num_assigned_fragments', 'num_bias_bins', 'num_bootstraps',
       'num_compatible_fragments', 'num_consistent_mappings',
       'num_inconsistent_mappings', 'num_libraries', 'num_mapped',
       'num_processed', 'num_targets', 'nvars_used', 'pct_counts_endogenous',
       'pc

In [6]:
#Filter cells to keep only day 0–day 3 samples, remove low-quality cells (total counts ≥ 1,000; 200 ≤ genes expressed ≤ 8,000; mitochondrial fraction ≤ 20%).
adata_iPSC_filter_day = adata_iPSC[adata_iPSC.obs['day'].isin(['day0','day1', 'day2', 'day3'])]
adata_iPSC_filter = adata_iPSC_filter_day[
    (adata_iPSC_filter_day.obs['total_counts'] >= 1000) & 
    (adata_iPSC_filter_day.obs['total_features'] >= 200) & 
    (adata_iPSC_filter_day.obs['total_features'] <= 10000) & 
    (adata_iPSC_filter_day.obs['pct_counts_MT'] <= 20)
]

In [7]:
sc.pp.normalize_total(adata_iPSC_filter, target_sum=1e4)
sc.pp.log1p(adata_iPSC_filter)


/work/home/cryoem666/miniconda3/envs/my_state/lib/python3.12/site-packages/scanpy/preprocessing/_normalization.py:269: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


In [62]:
# 3.1 强制包含的标记基因列表
marker_genes = [
    "NANOG", "PDGFA", "MT1X", "SPP1", "KCNS3", "WIPF3", "METTL13", "IGSF21", "NMU", "AKAP12",
    "TGFBI", "MED14", "KCNQ2", "FOS", "MT1G", "PSMA7", "USP22", "WDR76", "HIST1H4B", "CHP1",
    "RPL41P1", "PTPRK","ASB16","GAREML"
]


# 3.2 检查哪些标记基因在数据集中
available_markers = [gene for gene in marker_genes if gene in [item.split('_')[1] for item in list(adata_iPSC_filter.var_names)]]
missing_markers = [gene for gene in marker_genes if gene not in [item.split('_')[1] for item in list(adata_iPSC_filter.var_names)]]

print(f"找到的标记基因: {len(available_markers)}/{len(marker_genes)}")
if missing_markers:
    print(f"缺失的标记基因: {missing_markers}")


找到的标记基因: 24/24


In [8]:
n_top_genes = 500
print(f"\n选择 {n_top_genes} 个高变基因...")

# 计算高变基因
sc.pp.highly_variable_genes(
    adata_iPSC_filter,
    n_top_genes=n_top_genes
)


highly_variable_genes = adata_iPSC_filter.var_names[adata_iPSC_filter.var['highly_variable']]
print(f"最终高变基因数量: {len(highly_variable_genes)}")

# 3.5 创建高变基因子集
adata_hvg = adata_iPSC_filter[:, highly_variable_genes].copy()
print(f"高变基因子集形状: {adata_hvg.shape}")


选择 500 个高变基因...
最终高变基因数量: 500
高变基因子集形状: (16234, 500)


In [8]:
set(adata_iPSC_filter.obs['day'])

{'day0', 'day1', 'day2', 'day3'}

In [9]:
# 4.1 首先筛选只包含day 0和day 3的细胞用于训练
print("\n=== 准备训练数据 ===")
day_03_mask = adata_hvg.obs['day'].isin(['day0','day3'])
adata_day_03 = adata_hvg[day_03_mask].copy()
print(f"Day 0和Day 3总细胞数: {adata_day_03.n_obs}")

# 4.2 从day 0和day 3各抽取1200个细胞用于训练
train_cells = []
for day_value in ['day0', 'day3']:
    day_cells = adata_day_03.obs_names[adata_day_03.obs['day'] == day_value]
    
    if len(day_cells) >= 1200:
        # 随机抽取1200个细胞
        selected = np.random.choice(day_cells, size=1200, replace=False)
    else:
        # 如果不足1200，使用所有细胞（可能有替换）
        print(f"警告: Day {day_value} 只有 {len(day_cells)} 个细胞，将使用所有细胞")
        selected = day_cells
    
    train_cells.extend(selected)

# 创建训练集
adata_train = adata_hvg[train_cells].copy()
print(f"训练集大小: {adata_train.n_obs}个细胞")
print("训练集day分布:")
print(adata_train.obs['day'].value_counts().sort_index())

# 4.3 准备评估数据（day 0-3每天600个细胞）
print("\n=== 准备评估数据 ===")
eval_cells = []
for day_value in ['day0','day1','day2','day3']:  # day 0-3
    day_mask = adata_hvg.obs['day'] == day_value
    
    # 排除已用于训练的细胞
    day_cells = adata_hvg.obs_names[day_mask]
    day_cells = [cell for cell in day_cells if cell not in train_cells]
    
    if len(day_cells) >= 600:
        # 随机抽取600个细胞
        selected = np.random.choice(day_cells, size=600, replace=False)
    else:
        # 如果不足600，使用所有剩余细胞
        print(f"警告: Day {day_value} 只有 {len(day_cells)} 个可用细胞用于评估")
        selected = day_cells
    
    eval_cells.extend(selected)

# 创建评估集
adata_eval = adata_hvg[eval_cells].copy()
print(f"评估集大小: {adata_eval.n_obs}个细胞")
print("评估集day分布:")
print(adata_eval.obs['day'].value_counts().sort_index())

# 4.4 可选：创建测试集（如果还需要测试数据）
print("\n=== 准备测试数据 ===")
test_cells = []
for day_value in ['day0','day1','day2','day3']:  # day 0-3
    day_mask = adata_hvg.obs['day'] == day_value
    
    # 排除已用于训练的细胞
    day_cells = adata_hvg.obs_names[day_mask]
    day_cells = [cell for cell in day_cells if cell not in train_cells and cell not in eval_cells]
    
    if len(day_cells) >= 600:
        # 随机抽取600个细胞
        selected = np.random.choice(day_cells, size=600, replace=False)
    else:
        # 如果不足600，使用所有剩余细胞
        print(f"警告: Day {day_value} 只有 {len(day_cells)} 个可用细胞用于评估")
        selected = day_cells
    
    test_cells.extend(selected)

# 创建评估集
adata_test = adata_hvg[test_cells].copy()
print(f"测试集大小: {adata_test.n_obs}个细胞")
print("测试集day分布:")
print(adata_test.obs['day'].value_counts().sort_index())

#


=== 准备训练数据 ===
Day 0和Day 3总细胞数: 7710
训练集大小: 2400个细胞
训练集day分布:
day
day0    1200
day3    1200
Name: count, dtype: int64

=== 准备评估数据 ===
评估集大小: 2400个细胞
评估集day分布:
day
day0    600
day1    600
day2    600
day3    600
Name: count, dtype: int64

=== 准备测试数据 ===
测试集大小: 2400个细胞
测试集day分布:
day
day0    600
day1    600
day2    600
day3    600
Name: count, dtype: int64


In [10]:
# Now adata_train, adata_eval, and adata_test are ready for use.
# Save them 
adata_train.write_h5ad("/work/home/cryoem666/xyf/temp/pycharm/Squidiff/data/diff_data/adata_train_500_gene.h5ad")
adata_eval.write_h5ad("/work/home/cryoem666/xyf/temp/pycharm/Squidiff/data/diff_data/adata_eval_500_gene.h5ad")
adata_test.write_h5ad("/work/home/cryoem666/xyf/temp/pycharm/Squidiff/data/diff_data/adata_test_500_gene.h5ad")